# Better Interface
We have replaced the User Interface of the first assignment with a nicer user interface developed with a Jupyter Notebook.

In particular we want the user interface to be able of:
* starting / stopping the robot “random position” behaviour by using two Buttons
* setting the linear and angular velocity by using two Sliders
* directly controlling the robot movements by using 5 Buttons, (forward, turn right, backward, turn left, stop)

## [Start/Stop] buttons
We created two buttons: 
* Start: for starting the random postion movement of the robot.
* Stop: for stopping the random position movement of the robot.

In [1]:
import ipywidgets as widgets
from geometry_msgs.msg import Twist
from matplotlib import animation, rc
from nav_msgs.msg import Odometry
import rospy
import jupyros as jr
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from rt2_assignment1.srv import Command

%matplotlib widget

## Initializing nodes ##
rospy.init_node('better_interface')
# Defining /user_interface client 
ui_client = rospy.ServiceProxy('/user_interface', Command)
# Defining /cmd_vel publisher
pub_ = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
 
## Create buttons and output screen ##
button1 = widgets.Button(description='Start')
button2 = widgets.Button(description='Stop')
out = widgets.Output()

## Buttons on_click call backs ##
def on_button1_clicked(_):
    with out:
        ui_client("start")
        print('start random moving...')
        
def on_button2_clicked(_):
    with out:
        ui_client("stop")
        print('Stop random moving')


button1.on_click(on_button1_clicked)
button2.on_click(on_button2_clicked)

# displaying button and its output together
widgets.VBox([HBox([button1,button2]), out])


## Velocity Sliders
* For setting the linear and angular velocity of the robot.
* *Note1*: I added a third slider for (Kp) the angular velocity multiplier which is used in *goToPoint* node.
* *Note2*: When the robot is in the random motion mode, we use the slider **Kp_a** for changing the angular velocity.
* *Note3*: When the robot is in the free motion mode, we use the slider **Angular Velocity** for changing the angular velocity.

In [2]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from matplotlib import animation, rc
import ipywidgets as widgets

## Defining 3 Sliders ##
# For linear velocity
linear = widgets.FloatSlider(value=0.3, min=0.1, max=0.6)
a1 = widgets.HBox([widgets.Label(value="Linear Velocity"), linear])
display(a1)

# For angular velocity
angular = widgets.FloatSlider(value=0.3, min=0.1, max=0.6)
a2 = widgets.HBox([widgets.Label(value="Angular Velocity"), angular])
display(a2)

# For angular multiplier Kp_a
kp = widgets.FloatSlider(value=3, min=0.1, max=5.0)
a3 = widgets.HBox([widgets.Label(value="Kp_a"), kp])
display(a3)

## Sliders' Change call_backs ##
# Here we used 3 global parameters for setting the velocities
def linear_value_change(change):
    l_vel = change['new']
    rospy.set_param("linear_vel",l_vel)
    
def angular_value_change(change):
    a_vel = change['new']
    rospy.set_param("angular_vel",a_vel)    

def kp_value_change(change):
    kp_vel = change['new']
    rospy.set_param("kp_vel",kp_vel)    
    
linear.observe(linear_value_change, names='value')
angular.observe(angular_value_change, names='value')
kp.observe(kp_value_change, names='value')


## Free movement
We can freely control the movement of the robot by using five buttons;
(forward, turn right, backward, turn left, stop)

In [3]:
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import ipywidgets as widgets

## Defining 5 buttons ##
up = Button(
layout=Layout(width='auto', align="center", grid_area='up'),
style=ButtonStyle(button_color='lightblue'),
           icon='fa-arrow-up')

down = Button(
layout=Layout(width='auto', grid_area='down'),
style=ButtonStyle(button_color='lightblue'),
           icon='fa-arrow-down')

right = Button(
layout=Layout(width='auto', grid_area='right'),
style=ButtonStyle(button_color='lightblue'),
           icon='fa-arrow-right')

left = Button(
layout=Layout(width='auto', grid_area='left'),
style=ButtonStyle(button_color='lightblue'),
           icon='fa-arrow-left')

stop = Button(
layout=Layout(width='auto', grid_area='stop'),
style=ButtonStyle(button_color='lightblue'),
           icon='fa-stop-circle')

# Defining Outputs
output = widgets.Output()

# Setting Buttons's positions
a = GridBox(children=[up, down, right, left,stop],
    layout=Layout(
        width='20%',
        grid_template_rows='auto auto',
        grid_template_columns='33% 33% 33%',
        grid_template_areas='''
        " . up . "
        "left stop right "
        ". down ."
        ''')
    )

# Displaying buttons and output screen
display(a)
display(output)


GridBox(children=(Button(icon='arrow-up', layout=Layout(grid_area='up', width='auto'), style=ButtonStyle(butto…

Output()

In [4]:
# Define cmd_vel publisher for setting velocities 
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
msg = Twist()

## Buttons on_click handles ##
def up_clicked(b):
    global msg
    msg.angular.z = 0
    msg.linear.x = rospy.get_param("linear_vel")
    pub.publish(msg)    
    with output:
        print("Forward")
        
def down_clicked(b):
    global msg
    msg.angular.z = 0
    msg.linear.x = rospy.get_param("linear_vel")*(-1)
    pub.publish(msg)  
    with output:
        print("Backword")
        
def left_clicked(b):
    global msg
    msg.angular.z = rospy.get_param("angular_vel")*(-1)
    msg.linear.x = 0
    pub.publish(msg)
    with output:
        print("Turn Left")
        
def right_clicked(b):
    global msg
    msg.angular.z = rospy.get_param("angular_vel")
    msg.linear.x = 0
    pub.publish(msg)
    with output:
        print("Trun Right")
        
def stop_clicked(b):
    global msg
    msg.linear.x = 0
    msg.angular.z = 0
    pub.publish(msg)
    with output:
        print("Stop")
        
up.on_click(up_clicked)
down.on_click(down_clicked)
left.on_click(left_clicked)
right.on_click(right_clicked)
stop.on_click(stop_clicked)